In [ ]:
import ROOT

pen the rootfile and get the workspace from the exercise_0

In [ ]:
fInput = ROOT.TFile("Workspace_mumufit.root")
ws = fInput.Get("ws")
ws.Print()

ou can set constant parameters that are known<br>
f you leave them floating, the fit procedure will determine their uncertainty<br>
ight now we will fix all the nuisance parameters just to speed up the computing time

In [ ]:
ws.var("meanJpsi").setConstant(1)
ws.var("sigmaJpsi").setConstant(1)
ws.var("alphaJpsi").setConstant(1)
ws.var("nJpsi").setConstant(1)
ws.var("NJpsi").setConstant(1)
ws.var("meanpsi2S").setConstant(1)
ws.var("Nbkg").setConstant(1)
ws.var("a1").setConstant(1)
ws.var("a2").setConstant(1)
ws.var("a3").setConstant(1)

onfigure the model, we need both the S+B and the B only models

In [ ]:
sbModel = ROOT.RooStats.ModelConfig()
sbModel.SetWorkspace(ws)
sbModel.SetPdf("totPDF")
sbModel.SetName("S+B Model")
poi = ROOT.RooArgSet(ws.var("cross_psi"))
poi.find("cross_psi").setRange(0.,40.)  #this is mostly for plotting
sbModel.SetParametersOfInterest(poi)

In [ ]:
bModel = sbModel.Clone()
bModel.SetPdf("totPDF")
bModel.SetName( sbModel.GetName() + "_with_poi_0")
poi.find("cross_psi").setVal(0)
bModel.SetSnapshot(poi)

irst example is with a frequentist approach

In [ ]:
fc = ROOT.RooStats.FrequentistCalculator(ws.data("data"), bModel, sbModel)
fc.SetToys(2500,1500)

reate hypotest inverter passing the desired calculator 

In [ ]:
calc = ROOT.RooStats.HypoTestInverter(fc)

et confidence level (e.g. 95% upper limits)

In [ ]:
calc.SetConfidenceLevel(0.95)

se CLs

In [ ]:
calc.UseCLs(1)

educe the noise

In [ ]:
calc.SetVerbose(0)

onfigure ToyMC Samler

In [ ]:
toymcs = calc.GetHypoTestCalculator().GetTestStatSampler()

se profile likelihood as test statistics 

In [ ]:
profll = ROOT.RooStats.ProfileLikelihoodTestStat(sbModel.GetPdf())

or CLs (bounded intervals) use one-sided profile likelihood

In [ ]:
profll.SetOneSided(1)

et the test statistic to use for toys

In [ ]:
toymcs.SetTestStatistic(profll)

In [ ]:
npoints = 8 #Number of points to scan
# min and max for the scan (better to choose smaller intervals)
poimin = poi.find("cross_psi").getMin()
poimax = poi.find("cross_psi").getMax()

In [ ]:
print "Doing a fixed scan  in interval : ", poimin, " , ", poimax
calc.SetFixedScan(npoints,poimin,poimax);

In [ ]:
result = calc.GetInterval() #This is a HypoTestInveter class object
upperLimit = result.UpperLimit()

xample using the BayesianCalculator<br>
ow we also need to specify a prior in the ModelConfig<br>
o be quicker, we'll use the PDF factory facility of RooWorkspace<br>
areful! For simplicity, we are using a flat prior, but this doesn't mean it's the best choice!

In [ ]:
ws.factory("Uniform::prior(cross_psi)")
sbModel.SetPriorPdf(ws.pdf("prior"))

onstruct the bayesian calculator

In [ ]:
bc = ROOT.RooStats.BayesianCalculator(ws.data("data"), sbModel)
bc.SetConfidenceLevel(0.95)
bc.SetLeftSideTailFraction(0.) # for upper limit

In [ ]:
bcInterval = bc.GetInterval()

ow let's print the result of the two methods<br>
irst the CLs

In [ ]:
print "################"
print "The observed CLs upper limit is: ", upperLimit

ompute expected limit

In [ ]:
print "Expected upper limits, using the B (alternate) model : "
print " expected limit (median) ", result.GetExpectedUpperLimit(0)
print " expected limit (-1 sig) ", result.GetExpectedUpperLimit(-1)
print " expected limit (+1 sig) ", result.GetExpectedUpperLimit(1)
print "################"

ow let's see what the bayesian limit is

In [ ]:
print "Bayesian upper limit on cross_psi = ", bcInterval.UpperLimit()

lot now the result of the scan 

irst the CLs

In [ ]:
freq_plot = ROOT.RooStats.HypoTestInverterPlot("HTI_Result_Plot","Frequentist scan result for psi xsec",result)
#Then the Bayesian posterior
bc_plot = bc.GetPosteriorPlot()

lot in a new canvas with style

In [ ]:
dataCanvas = ROOT.TCanvas("dataCanvas")
dataCanvas.Divide(2,1)
dataCanvas.SetLogy(0)
dataCanvas.cd(1)
freq_plot.Draw("2CL")
dataCanvas.cd(2)
bc_plot.Draw()
dataCanvas.SaveAs("exercise_3.png")